https://cloud.google.com/ml-engine/docs/command-line
Create a project and enable it


apt-get install sudo curl and install gcloud

In [ ]:
https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
# Create an environment variable for the correct distribution
export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)"

# Add the Cloud SDK distribution URI as a package source
echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list

# Import the Google Cloud Platform public key
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

# Update the package list and install the Cloud SDK
sudo apt-get update && sudo apt-get install google-cloud-sdk

In [ ]:
init gcloud ans set project and authenticate

In [ ]:
gcloud init
cloud auth application-default login
gcloud config set project ai-ml-dl


In [ ]:
https://cloud.google.com/ml-engine/docs/getting-started-training-prediction

Export ml,  Model validation failed: Serving metagraph must contain exactly one SignatureDef with key: serving_default

In [5]:
#Input and output shape
print(model.input)
print(model.output)

Tensor("gru_1_input:0", shape=(?, 100, 55), dtype=float32)
Tensor("dense_1/Softmax:0", shape=(?, 55), dtype=float32)


In [ ]:
Make bucket, upload model and depoy
MODEL_NAME=BeirasRnn
gcloud ml-engine models create $MODEL_NAME --enable-logging
gsutil mb gs://beiras_rnn/
    
    
gsutil cp -r ./export-tf/2 gs://beiras_rnn/export-google-ml/7
MODEL_NAME=BeirasRnn
DEPLOYMENT_SOURCE="gs://beiras_rnn/export-google-ml/7"
gcloud ml-engine versions create "v07" --model $MODEL_NAME  --origin $DEPLOYMENT_SOURCE



Install google api lib for python
pip install --upgrade google-api-python-client

Predict from ml
the input shape musth hava a shape (number_chars,window_size), i do not now why

In [13]:
import googleapiclient.discovery
import sys
import numpy as np
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"
# Load values
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# Clean the text
input_clean=clean_text(input_init.lower())
input_clean = input_clean[:window_size]
# Text to array [1,input_lenght,num_chars]
#x_test = np.zeros((1,window_size, number_chars))
#for t, char in enumerate(input_clean):
#    x_test[0, t, chars_to_indices[char]] = 1.
#x_test   
x_test = np.zeros((number_chars,window_size))
for t, char in enumerate(input_clean):
    x_test[ chars_to_indices[char],t] = 1.
    
project="ai-ml-dl"
model="BeirasRnn"
version="v07"
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(project, model)
if version is not None:
        name += '/versions/{}'.format(version)
instances={'sequence':x_test.tolist()}
response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
if 'error' in response:
    raise RuntimeError(response['error'])
test_predict=np.array(response['predictions'][0]['scores'])
r = np.argmax(test_predict)  # predict class of each test input
print(test_predict)
print(r,indices_to_chars[r])

[  3.02791549e-03   1.82074245e-13   1.66169353e-04   7.92613406e-08
   5.73130601e-05   1.31240768e-05   2.25630544e-07   1.25827592e-05
   3.36391054e-06   2.49683217e-06   2.67709061e-06   2.58707291e-06
   3.85522708e-06   1.68106453e-06   9.78884920e-02   7.72795000e-04
   1.28873562e-05   6.51064329e-05   4.04933803e-02   5.68993273e-05
   7.88223770e-05   2.52500555e-04   7.03036606e-01   2.65130211e-06
   2.58586806e-05   6.67042732e-02   1.81544849e-04   4.20611771e-03
   6.22344902e-03   1.42863850e-04   1.35668085e-06   8.01028404e-03
   6.67816699e-02   1.40305041e-04   2.27522425e-04   2.22166782e-05
   3.05614267e-05   1.81408183e-07   9.27245637e-05   1.32501589e-06
   1.63243293e-13   1.66929716e-13   1.00698369e-03   2.05680620e-13
   6.43400635e-05   1.30786441e-13   1.48362204e-04   1.64503529e-13
   2.37581412e-06   1.64584519e-13   1.40918200e-05   1.60686067e-13
   1.50571059e-05   8.69439134e-07   1.57377144e-06]
22 i


Predict from console, return a score

In [14]:
#Save the file
import json
with open('data.json', 'w') as fp:
    json.dump(instances, fp)

gcloud ml-engine predict --model=BeirasRnn --json-instances=data.json --version=v07


Predict with funcions, fail



In [17]:
# Define a function
def predict_one(text_predict,service,model_name,window_size,chars_to_indices, indices_to_chars):
    # Convert input sequence to array
    number_chars=len(chars_to_indices)
    x_test = np.zeros((number_chars,window_size))
    for t, char in enumerate(text_predict):
        x_test[chars_to_indices[char],t] = 1.
    #Prepare the request
    instances={'sequence':x_test.tolist()}
    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
    if 'error' in response:
        raise RuntimeError(response['error'])
    test_predict=np.array(response['predictions'][0]['scores'])
    r = np.argmax(test_predict)  # predict class of each test input
    return (indices_to_chars[r])

In [19]:

# Complete a sequence using the server
def predict_window(text_predict,number_predict,window_size,lproject,lmodel,lversion):
    
    # Get dictionaries
    chars_to_indices, indices_to_chars = load_coded_dictionaries()
    # Clean the test
    input_clean=clean_text(text_predict.lower())
    # Get stub
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(lproject, lmodel)
    if lversion is not None:
        name += '/versions/{}'.format(lversion)
    print(name)
    # Call server for all charazters
    for i in range(number_predict):
        d=predict_one(input_clean[i:],service,name,window_size,chars_to_indices, indices_to_chars)
        input_clean+=d
    return input_clean

In [21]:
import googleapiclient.discovery
project="ai-ml-dl"
model="BeirasRnn"
version="v07"

input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"

window_size = 100
number_predict=20
      

response=predict_window(input_init[:window_size],number_predict,
                        window_size,project,model,version)
response

projects/ai-ml-dl/models/BeirasRnn/versions/v07


'se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcui  ie   eil  hllns e'